# Shallow for genres

## Imports

In [ ]:
from functools import cache
from ast import literal_eval
import os

import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

from nltk.tokenize import word_tokenize

from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression

from tqdm.auto import tqdm
tqdm.pandas()

from pymorphy3 import MorphAnalyzer
morph = MorphAnalyzer()

import re
pat = re.compile(r'(\n{3,})')
path = '/Users/v.i.zykova/PycharmProjects/NLPworkshop/fics_all'

## Data Preparation

In [ ]:
evals = ['author', 'fandom', 'characters', 'genre', 'events', 'series',
       'warnings', 'beta', 'translator']
drops = ['author_href', 'fandom_href', 'characters_href', 'events_href', 'series_href',
       'beta_href', 'collections_href', 'translator_href']
fics = pd.read_csv('metadata_fics_clean.csv', converters={col: literal_eval for col in evals})
fics = fics.drop(columns=drops)
fics['summary'] = fics.summary.fillna('')
fics = fics[(fics.n_words >= 2000) & (fics.n_words <= 5000)]
fics.head(1)

,id,error,name,type,summary,author,fandom,characters,rating,genre,...,warnings,beta,translator,size_type,MB,n_words,k_simbols,n_pages,date_from,date_to
10,110493,False,Все дороги ведут на Сваард,джен,"Калин появилась в Крайнсере — то бишь, в Крайн...",[Hioshidzuka],[Ориджиналы],[],PG-13,"[Фэнтези, Общий]",...,[],[],[],Мини,29,4543,30,14,2017/10/26,2017/10/26


In [ ]:
@cache
def analyse(word):
    return morph.parse(word)[0].normal_form

In [ ]:
def preprocess(text):
    clean = []
    for word in word_tokenize(text):
        if word.isalpha():
            clean.append(analyse(word))
    return ' '.join(clean)

In [ ]:
def clean_genres(line):
    line = line.replace('Hurt/comfort', 'Hurt-comfort')
    line = line.split('/')
    return line

In [ ]:
def read_fulltext(idx):
    p = [el for el in os.listdir(path + f'/{idx}') if '.txt' in el][0]
    with open(path + f'/{idx}/{p}', encoding='utf-8') as f:
        text = f.read()
    return text[pat.search(text).end():]

In [ ]:
l = '''Драма,Романтика,Юмор,Ангст,Фэнтези,Приключения,Hurt-comfort,Флафф,Мистика'''.split(',')

In [ ]:
# df = pd.DataFrame()
# df['id'] = fics['id']
# df['text'] = fics['name'] + ' ' + fics['summary']
# df = df.merge(tmp, on='id')

# df['full_text'] = df['id'].progress_apply(read_fulltext)
# df['text_clean'] = df.text.progress_apply(preprocess)
# df['clean_full_text'] = df.full_text.progress_apply(preprocess)

# df.to_csv('preproc_data.csv')

In [ ]:
df = pd.read_csv('preproc_data.csv', index_col=0, converters={'genre': literal_eval})
df = df.explode('genre')
df['genre'] = df.genre.apply(clean_genres)
df = df.explode('genre')
df = df[df.genre.isin(l)]
df = df.groupby(['id', 'text', 'full_text','text_clean', 'clean_full_text'], as_index=False).agg(list)
df.head()

,id,text,full_text,text_clean,clean_full_text,genre
0,22,Army of Darkness 5-ый курс Гарри Поттера в шко...,Пролог\n\nВесь этот июльский день — последний ...,army of darkness курс гарри поттер в школа чар...,пролог весь этот июльский день последний в мес...,"[Драма, Романтика, Приключения]"
1,39,Больше чем друзья Дружба между юношей и девушк...,Переводчик: Vivian\nИсточник: www.portkey.org\...,большой чем друг дружба между юноша и девушка ...,переводчик vivian источник фандом гарри поттер...,[Романтика]
2,40,"Люби меня, люблю тебя Реализация замыслов хоро...",Переводчик: Vivian\nИсточник: www.portkey.org\...,любить я любить ты реализация замысел хороший ...,переводчик vivian источник фандом гарри поттер...,"[Романтика, Юмор]"
3,41,Подарок на его день Рождения Неожиданные подар...,Переводчик: Vivian\nИсточник: www.portkey.org\...,подарок на он день рождение неожиданный подаро...,переводчик vivian источник фандом гарри поттер...,[Романтика]
4,72,Yesterday Начало или Конец? Жизнь или Смерть? ...,Глава 1. Гарри Поттер\n\nYesterday\nAll my tro...,yesterday начало или конец жизнь или смерть по...,глава гарри поттер yesterday all my troubles s...,[Романтика]


## Models

### Preproc

Train and test split

In [ ]:
train, test = train_test_split(df, test_size=0.2, shuffle=True, random_state=42)

Vectorizing all

In [ ]:
vectorizer = TfidfVectorizer()
mlb = MultiLabelBinarizer()

vecs_train = vectorizer.fit_transform(train.text_clean)
vecs_test = vectorizer.transform(test.text_clean)

lbl_train = mlb.fit_transform(train.genre)
lbl_test = mlb.transform(test.genre)

vectorizer_full = TfidfVectorizer()

vecs_train_full = vectorizer_full.fit_transform(train.clean_full_text)
vecs_test_full = vectorizer_full.transform(test.clean_full_text)

lbl_train.shape

(5678, 9)

### On descriptions and names

#### LogReg

In [ ]:
lr = LogisticRegression()
multi_target_lr = MultiOutputClassifier(lr, n_jobs=1)
multi_target_lr.fit(vecs_train, lbl_train)

MultiOutputClassifier(estimator=LogisticRegression(), n_jobs=1)

In [ ]:
preds_lr = multi_target_lr.predict(vecs_test)

In [ ]:
print(classification_report(lbl_test, preds_lr, target_names=mlb.classes_))

              precision    recall  f1-score   support

Hurt-comfort       0.00      0.00      0.00       131
       Ангст       1.00      0.01      0.01       285
       Драма       0.51      0.20      0.28       526
     Мистика       0.00      0.00      0.00       109
 Приключения       0.00      0.00      0.00       121
   Романтика       0.65      0.15      0.24       379
       Флафф       0.00      0.00      0.00       141
     Фэнтези       0.00      0.00      0.00       170
        Юмор       0.67      0.04      0.07       324

   micro avg       0.56      0.08      0.14      2186
   macro avg       0.31      0.04      0.07      2186
weighted avg       0.46      0.08      0.12      2186
 samples avg       0.12      0.09      0.10      2186



/Users/v.i.zykova/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/v.i.zykova/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


#### KNN

In [ ]:
knn = KNeighborsClassifier()
multi_target_knn = MultiOutputClassifier(knn, n_jobs=1)
multi_target_knn.fit(vecs_train, lbl_train)

MultiOutputClassifier(estimator=KNeighborsClassifier(), n_jobs=1)

In [ ]:
preds_knn = multi_target_knn.predict(vecs_test)

In [ ]:
print(classification_report(lbl_test, preds_knn, target_names=mlb.classes_))

              precision    recall  f1-score   support

Hurt-comfort       0.17      0.02      0.03       131
       Ангст       0.30      0.13      0.18       285
       Драма       0.45      0.41      0.43       526
     Мистика       0.38      0.07      0.12       109
 Приключения       0.19      0.03      0.06       121
   Романтика       0.41      0.31      0.36       379
       Флафф       0.25      0.02      0.04       141
     Фэнтези       0.32      0.04      0.07       170
        Юмор       0.49      0.13      0.20       324

   micro avg       0.41      0.20      0.27      2186
   macro avg       0.33      0.13      0.17      2186
weighted avg       0.37      0.20      0.24      2186
 samples avg       0.27      0.21      0.22      2186



/Users/v.i.zykova/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


#### Random Forest

In [ ]:
forest = RandomForestClassifier(random_state=1)
multi_target_forest = MultiOutputClassifier(forest, n_jobs=1)
multi_target_forest.fit(vecs_train, lbl_train)

MultiOutputClassifier(estimator=RandomForestClassifier(random_state=1),
                      n_jobs=1)

In [ ]:
preds_rf = multi_target_forest.predict(vecs_test)

In [ ]:
print(classification_report(lbl_test, preds_rf, target_names=mlb.classes_))

              precision    recall  f1-score   support

Hurt-comfort       0.00      0.00      0.00       131
       Ангст       0.08      0.00      0.01       285
       Драма       0.54      0.16      0.25       526
     Мистика       0.62      0.07      0.13       109
 Приключения       0.00      0.00      0.00       121
   Романтика       0.56      0.17      0.27       379
       Флафф       0.00      0.00      0.00       141
     Фэнтези       0.89      0.05      0.09       170
        Юмор       0.38      0.01      0.02       324

   micro avg       0.53      0.08      0.14      2186
   macro avg       0.34      0.05      0.08      2186
weighted avg       0.39      0.08      0.12      2186
 samples avg       0.12      0.08      0.09      2186



/Users/v.i.zykova/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/v.i.zykova/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


### Full texts

#### LogReg

In [ ]:
lr = LogisticRegression()
multi_target_lr = MultiOutputClassifier(lr, n_jobs=1)
multi_target_lr.fit(vecs_train_full, lbl_train)

MultiOutputClassifier(estimator=LogisticRegression(), n_jobs=1)

In [ ]:
preds_lr = multi_target_lr.predict(vecs_test_full)

In [ ]:
print(classification_report(lbl_test, preds_lr, target_names=mlb.classes_))

              precision    recall  f1-score   support

Hurt-comfort       0.00      0.00      0.00       131
       Ангст       0.60      0.09      0.15       285
       Драма       0.62      0.40      0.49       526
     Мистика       1.00      0.01      0.02       109
 Приключения       0.00      0.00      0.00       121
   Романтика       0.69      0.38      0.49       379
       Флафф       0.00      0.00      0.00       141
     Фэнтези       0.92      0.07      0.13       170
        Юмор       0.77      0.25      0.38       324

   micro avg       0.67      0.22      0.33      2186
   macro avg       0.51      0.13      0.18      2186
weighted avg       0.58      0.22      0.29      2186
 samples avg       0.30      0.24      0.25      2186



/Users/v.i.zykova/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/v.i.zykova/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


#### KNN

In [ ]:
knn = KNeighborsClassifier()
multi_target_knn = MultiOutputClassifier(knn, n_jobs=1)
multi_target_knn.fit(vecs_train_full, lbl_train)

MultiOutputClassifier(estimator=KNeighborsClassifier(), n_jobs=1)

In [ ]:
preds_knn = multi_target_knn.predict(vecs_test_full)

In [ ]:
print(classification_report(lbl_test, preds_knn, target_names=mlb.classes_))

              precision    recall  f1-score   support

Hurt-comfort       0.31      0.08      0.13       131
       Ангст       0.29      0.13      0.18       285
       Драма       0.47      0.38      0.42       526
     Мистика       0.29      0.05      0.08       109
 Приключения       0.40      0.05      0.09       121
   Романтика       0.37      0.58      0.46       379
       Флафф       0.13      0.11      0.12       141
     Фэнтези       0.65      0.13      0.22       170
        Юмор       0.43      0.10      0.16       324

   micro avg       0.38      0.25      0.30      2186
   macro avg       0.37      0.18      0.21      2186
weighted avg       0.39      0.25      0.27      2186
 samples avg       0.31      0.26      0.27      2186



/Users/v.i.zykova/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


#### Random Forest

In [ ]:
forest = RandomForestClassifier(random_state=1)
multi_target_forest = MultiOutputClassifier(forest, n_jobs=1)
multi_target_forest.fit(vecs_train_full, lbl_train)

MultiOutputClassifier(estimator=RandomForestClassifier(random_state=1),
                      n_jobs=1)

In [ ]:
preds_rf = multi_target_forest.predict(vecs_test_full)

In [ ]:
print(classification_report(lbl_test, preds_rf, target_names=mlb.classes_))

              precision    recall  f1-score   support

Hurt-comfort       0.00      0.00      0.00       131
       Ангст       0.80      0.01      0.03       285
       Драма       0.60      0.18      0.28       526
     Мистика       0.00      0.00      0.00       109
 Приключения       0.00      0.00      0.00       121
   Романтика       0.77      0.16      0.26       379
       Флафф       0.00      0.00      0.00       141
     Фэнтези       1.00      0.06      0.12       170
        Юмор       1.00      0.04      0.07       324

   micro avg       0.68      0.08      0.15      2186
   macro avg       0.46      0.05      0.08      2186
weighted avg       0.61      0.08      0.14      2186
 samples avg       0.13      0.09      0.10      2186



/Users/v.i.zykova/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/v.i.zykova/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
